In [34]:
from nba_api.stats import endpoints
import plotly.express as px
import pandas as pd

In [42]:
# Fetch NBA standings data
standings_full = endpoints.leaguestandings.LeagueStandings().get_data_frames()[0]

# Limit to relevant columns
col = ['TeamCity', 'TeamName', 'WINS', 'LOSSES']
standings = standings_full[col]

# Create Team Owner Dict for wins pool
owner_map = {
    'Celtics': 'Eric',
    'Nuggets': 'Divya',
    'Mavericks': 'Divya',
    '76ers': 'Zach',
    'Warriors': 'Kent',
    'Bucks': 'Nehemiah',
    'Hawks': 'Eric',
    'Timberwolves': 'Nehemiah',
    'Pelicans': 'Divya',
    'Pacers': 'Richard',
    'Magic': 'Cuente',
    'Thunder': 'Nehemiah',
    'Cavaliers': 'Cuente',
    'Clippers': 'Zach',
    'Rockets': 'N/A',
    'Nets': 'Divya',
    'Knicks': 'Kent',
    'Spurs': 'Kent',
    'Lakers': 'Cuente',
    'Heat': 'Richard',
    'Suns': 'Richard',
    'Raptors': 'Nehemiah',
    'Bulls': 'Kent',
    'Trail Blazers': 'Eric',
    'Kings': 'Eric',
    'Hornets': 'Richard',
    'Jazz': 'Zach',
    'Pistons': 'Cuente',
    'Grizzlies': 'Zach',
    'Wizards': 'N/A'
}

# Assign Wins Pool Owners
standings['Owner'] = standings['TeamName'].map(owner_map)

# Create over/under Dict for wins pool
ou_map = {
    'Celtics': 54.5,
    'Nuggets': 52.5,
    'Mavericks': 45.5,
    '76ers': 48.5,
    'Warriors': 48.5,
    'Bucks': 54.5,
    'Hawks': 42.5,
    'Timberwolves': 44.5,
    'Pelicans': 44.5,
    'Pacers': 38.5,
    'Magic': 37.5,
    'Thunder': 44.5,
    'Cavaliers': 50.5,
    'Clippers': 46.5,
    'Rockets': 31.5,
    'Nets': 37.5,
    'Knicks': 45.5,
    'Spurs': 28.5,
    'Lakers': 47.5,
    'Heat': 45.5,
    'Suns': 51.5,
    'Raptors': 36.5,
    'Bulls': 37.5,
    'Trail Blazers': 28.5,
    'Kings': 44.5,
    'Hornets': 31.5,
    'Jazz': 35.5,
    'Pistons': 28.5,
    'Grizzlies': 45.5,
    'Wizards': 24.5
}

# Create color Dict for wins pool
color_map = {
    'Celtics': '#007A33',
    'Nuggets': '#0E2240',
    'Mavericks': '#00538C',
    '76ers': '#006BB6',
    'Warriors': '#1D428A',
    'Bucks': '#00471B',
    'Hawks': '#E03A3E',
    'Timberwolves': '#0C2340',
    'Pelicans': '#0C2340',
    'Pacers': '#002D62',
    'Magic': '#0077C0',
    'Thunder': '#007AC1',
    'Cavaliers': '#860038',
    'Clippers': '#C8102E',
    'Rockets': '#CE1141',
    'Nets': '#000000',
    'Knicks': '#F58426',
    'Spurs': '#C4CED4',
    'Lakers': '#552583',
    'Heat': '#98002E',
    'Suns': '#E56020',
    'Raptors': '#CE1141',
    'Bulls': '#CE1141',
    'Trail Blazers': '#E03A3E',
    'Kings': '#5A2D81',
    'Hornets': '#00788C',
    'Jazz': '#002B5C',
    'Pistons': '#C8102E',
    'Grizzlies': '#5D76A9',
    'Wizards': '#002B5C'
}


# Assign Wins Pool Owners
standings['O/U'] = standings['TeamName'].map(ou_map)

# Calculate Points Per Team
standings['Points'] = standings.apply(lambda row: row['WINS'] + 2 * max(row['WINS'] - row['O/U'], 0), axis=1)

# Drop Teams With No Owner
standings = standings[standings['Owner'] != 'N/A']

# Calculate Total Points Per Owner
total_points = standings.groupby(['Owner'])['Points'].sum().reset_index()
total_points.rename({'Points': 'TotalPoints'}, axis=1, inplace=True)
standings = standings.merge(total_points, on='Owner')

# Sort Standings
standings = standings.sort_values(by=['TotalPoints', 'Owner'], ascending=False)

/var/folders/84/sb4dr76n06x6zw0p7llk4gpc0000gn/T/ipykernel_81939/1520873204.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/84/sb4dr76n06x6zw0p7llk4gpc0000gn/T/ipykernel_81939/1520873204.py:115: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/84/sb4dr76n06x6zw0p7llk4gpc0000gn/T/ipykernel_81939/1520873204.py:118: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [43]:
# View Standings df
standings.head()

,TeamCity,TeamName,WINS,LOSSES,Owner,O/U,Points,TotalPoints
0,Denver,Nuggets,8,1,Divya,52.5,8,22
1,Dallas,Mavericks,6,2,Divya,45.5,6,22
2,Brooklyn,Nets,4,4,Divya,37.5,4,22
3,New Orleans,Pelicans,4,4,Divya,44.5,4,22
8,Milwaukee,Bucks,5,2,Nehemiah,54.5,5,19


In [44]:
# Plot

# Stacked bar chart using Plotly Express
fig = px.bar(standings,
             x='Owner',
             y='Points',
             color='TeamName',
             color_discrete_map=color_map,
             title='Wins Pool Points',
             barmode='stack',
             text='TeamName')

fig.update_traces(texttemplate='%{text}', textposition='inside', showlegend=False)

for i, owner in enumerate(total_points['Owner']):
    total = total_points['TotalPoints'][i]
    fig.add_annotation(x=owner, y=total, text=str(total),
                       showarrow=False, font=dict(color='white'), yshift=10)

fig.show()

In [96]:
gamelog_full = endpoints.LeagueGameLog().get_data_frames()[0]
gamelog = gamelog_full[['TEAM_ABBREVIATION', 'GAME_DATE', 'WL']]

# Create Team Owner Dict for wins pool...again
owner_map_2 = {
    'ATL': 'Eric',
    'BKN': 'Divya',
    'BOS': 'Eric',
    'CHA': 'Richard',
    'CHI': 'Kent',
    'CLE': 'Cuente',
    'DAL': 'Divya',
    'DEN': 'Divya',
    'DET': 'Cuente',
    'GSW': 'Kent',
    'HOU': 'N/A',
    'IND': 'Richard',
    'LAC': 'Zach',
    'LAL': 'Cuente',
    'MEM': 'Zach',
    'MIA': 'Richard',
    'MIL': 'Nehemiah',
    'MIN': 'Nehemiah',
    'NOP': 'Divya',
    'NYK': 'Kent',
    'OKC': 'Nehemiah',
    'ORL': 'Cuente',
    'PHI': 'Zach',
    'PHX': 'Richard',
    'POR': 'Eric',
    'SAC': 'Eric',
    'SAS': 'Kent',
    'TOR': 'Nehemiah',
    'UTA': 'Zach',
    'WAS': 'N/A'

}

# Assign Wins Pool Owners
gamelog['Owner'] = gamelog['TEAM_ABBREVIATION'].map(owner_map_2)

# Wins Boolean
gamelog['WinMask'] = gamelog['WL'] == 'W'

# Change to Dates
gamelog['GAME_DATE'] = pd.to_datetime(gamelog['GAME_DATE'])

# Drop Teams With No Owner
gamelog = gamelog[gamelog['Owner'] != 'N/A']

# Pivot to get cumulative points
gamelog_pivot = gamelog.pivot_table(index='GAME_DATE', columns='Owner', values='WinMask', aggfunc='sum').cumsum().reset_index()

# Fill NA
gamelog_pivot.fillna(method='ffill', inplace=True)
gamelog_pivot.fillna(0, inplace=True)

/var/folders/84/sb4dr76n06x6zw0p7llk4gpc0000gn/T/ipykernel_81939/3469735499.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/84/sb4dr76n06x6zw0p7llk4gpc0000gn/T/ipykernel_81939/3469735499.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/84/sb4dr76n06x6zw0p7llk4gpc0000gn/T/ipykernel_81939/3469735499.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [100]:
# Season Long Graph
fig = px.line(gamelog_pivot,
             x='GAME_DATE',
             y=gamelog_pivot.columns[1:],
             title='Wins Pool Points By Date')

fig.update_layout(xaxis_title='Date',
                  yaxis_title='Points',
                  legend={'title': {'text': 'Owner'}})

fig.show()